In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from sklearn.datasets import load_iris
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split

In [ ]:
# Warstwa Dense - IRIS, dynamiczny batch size
iris = load_iris()
X = iris.data
y = iris.target.reshape(-1, 1)

encoder = OneHotEncoder(sparse_output=False)
y = encoder.fit_transform(y)

scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

model = Sequential(
    [
        Input(shape=(4,)),
        Dense(16, activation="relu"),
        Dense(3, activation="softmax"),
    ]
)

optimizer = tf.keras.optimizers.Adam()
loss_fn = tf.keras.losses.CategoricalCrossentropy()

epochs = 10
print("Trening..")

for epoch in range(epochs):
    dynamic_batch_size = np.random.randint(4, 32)

    train_dataset = (
        tf.data.Dataset.from_tensor_slices((X_train, y_train))
        .shuffle(len(X_train))
        .batch(dynamic_batch_size)
    )

    epoch_loss_avg = tf.keras.metrics.Mean()

    for x_batch, y_batch in train_dataset:
        with tf.GradientTape() as tape:
            logits = model(x_batch, training=True)
            loss_value = loss_fn(y_batch, logits)
        grads = tape.gradient(loss_value, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))
        epoch_loss_avg.update_state(loss_value)

    print(
        f"Epoka {epoch + 1}: Batch Size = {dynamic_batch_size}, Strata = {epoch_loss_avg.result().numpy():.4f}"
    )


Trening..
Epoka 1: Batch Size = 13, Strata = 1.0265
Epoka 2: Batch Size = 27, Strata = 0.9750
Epoka 3: Batch Size = 12, Strata = 0.9364
Epoka 4: Batch Size = 23, Strata = 0.9113
Epoka 5: Batch Size = 9, Strata = 0.8400
Epoka 6: Batch Size = 30, Strata = 0.8046
Epoka 7: Batch Size = 30, Strata = 0.7872
Epoka 8: Batch Size = 29, Strata = 0.7424
Epoka 9: Batch Size = 26, Strata = 0.7515
Epoka 10: Batch Size = 8, Strata = 0.7137


In [5]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Warstwa konwolucyjna Conv2D + batchnormalization (MINST)

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

model = models.Sequential(
    [
        layers.Input(shape=(28, 28, 1)),
        
        layers.Conv2D(32, (3, 3), use_bias=False),
        layers.BatchNormalization(),
        layers.Activation("relu"),
        layers.MaxPooling2D((2, 2)),
        
        layers.Conv2D(64, (3, 3), use_bias=False),
        layers.BatchNormalization(),
        layers.Activation("relu"),
        layers.MaxPooling2D((2, 2)),
        
        layers.Conv2D(64, (3, 3), use_bias=False),
        layers.BatchNormalization(),
        layers.Activation("relu"),
        
        # Classifier
        layers.Flatten(),
        layers.Dense(64, activation="relu"),
        layers.Dense(10, activation="softmax"),
    ]
)

model.compile(
    optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

model.summary()
model.fit(x_train, y_train, epochs=3, batch_size=64)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 26, 26, 32)     │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 26, 26, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 26, 26, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 11, 11, 64)     │        18,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 11, 11, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 11, 11, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 3, 3, 64)       │        36,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 3, 3, 64)       │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 3, 3, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 576)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 64)             │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 93,802 (366.41 KB)

 Trainable params: 93,482 (365.16 KB)

 Non-trainable params: 320 (1.25 KB)

Epoch 1/3
938/938 ━━━━━━━━━━━━━━━━━━━━ 29s 27ms/step - accuracy: 0.9323 - loss: 0.2265
Epoch 2/3
938/938 ━━━━━━━━━━━━━━━━━━━━ 27s 28ms/step - accuracy: 0.9889 - loss: 0.0358
Epoch 3/3
938/938 ━━━━━━━━━━━━━━━━━━━━ 25s 26ms/step - accuracy: 0.9927 - loss: 0.0238


In [6]:
import tensorflow as tf
from tensorflow.keras import layers, models, datasets, utils

max_features = 10000
maxlen = 200

print("Ładowanie danych IMDB...")

(x_train, y_train), (x_test, y_test) = datasets.imdb.load_data(num_words=max_features)

x_train = utils.pad_sequences(x_train, maxlen=maxlen)
x_test = utils.pad_sequences(x_test, maxlen=maxlen)

model = models.Sequential(
    [
        layers.Embedding(input_dim=max_features, output_dim=64, input_length=maxlen),
        # LSTM musi zwracać sekwencję (return_sequences=True),
        # aby kolejna warstwa rekurencyjna (GRU) miała co przetwarzać
        layers.LSTM(64, return_sequences=True, dropout=0.2, recurrent_dropout=0.2),
        # GRU przetwarza wynik z LSTM i zwraca ostateczny wektor
        layers.GRU(32, dropout=0.2),
        layers.Dense(1, activation="sigmoid"),
    ]
)

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

print("\nRozpoczynam naukę...")
history = model.fit(x_train, y_train, epochs=3, batch_size=64, validation_split=0.2)

final_acc = history.history["accuracy"][-1]
print(f"\nOstateczna dokładność: {final_acc:.4f}")


Ładowanie danych IMDB...

Rozpoczynam naukę...
Epoch 1/3


c:\Python311\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


313/313 ━━━━━━━━━━━━━━━━━━━━ 66s 194ms/step - accuracy: 0.6736 - loss: 0.5778 - val_accuracy: 0.7968 - val_loss: 0.4424
Epoch 2/3
313/313 ━━━━━━━━━━━━━━━━━━━━ 84s 199ms/step - accuracy: 0.8655 - loss: 0.3301 - val_accuracy: 0.8386 - val_loss: 0.3746
Epoch 3/3
313/313 ━━━━━━━━━━━━━━━━━━━━ 64s 206ms/step - accuracy: 0.8966 - loss: 0.2683 - val_accuracy: 0.8072 - val_loss: 0.4248

Ostateczna dokładność: 0.8866


In [7]:
import tensorflow as tf
from tensorflow.keras import layers, models, datasets, utils

# Warstwa Transformer
vocab_size = 10000
maxlen = 200
embed_dim = 64
num_heads = 4 

(x_train, y_train), (x_test, y_test) = datasets.imdb.load_data(num_words=vocab_size)
x_train = utils.pad_sequences(x_train, maxlen=maxlen)
x_test = utils.pad_sequences(x_test, maxlen=maxlen)

inputs = layers.Input(shape=(maxlen,))

# Warstwa Embedding
x = layers.Embedding(vocab_size, embed_dim)(inputs)

# Blok Rekurencyjny (LSTM + GRU)
x = layers.LSTM(64, return_sequences=True)(x)
x = layers.GRU(64, return_sequences=True)(x)

# Self-Attention
attention_output = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)(
    x, x
)

# Redukcja wymiarowości
# Zmiana z (Batch, Time, Features) na (Batch, Features)
x = layers.GlobalAveragePooling1D()(attention_output)


x = layers.Dropout(0.2)(x)
x = layers.Dense(32, activation="relu")(x)
outputs = layers.Dense(1, activation="sigmoid")(x)  

model = tf.keras.Model(inputs=inputs, outputs=outputs)


model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

model.summary()

print("\nRozpoczynam trening...")
model.fit(x_train, y_train, epochs=3, batch_size=64, validation_split=0.2)


Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4       │ (None, 200)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_2         │ (None, 200, 64)   │    640,000 │ input_layer_4[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_2 (LSTM)       │ (None, 200, 64)   │     33,024 │ embedding_2[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gru_2 (GRU)         │ (None, 200, 64)   │     24,960 │ lstm_2[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 200, 64)   │     66,368 │ gru_2[0][0],      │
│ (MultiHeadAttentio… │                   │            │ gru_2[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 64)        │          0 │ multi_head_atten… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 64)        │          0 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 32)        │      2,080 │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_7 (Dense)     │ (None, 1)         │         33 │ dense_6[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 766,465 (2.92 MB)

 Trainable params: 766,465 (2.92 MB)

 Non-trainable params: 0 (0.00 B)


Rozpoczynam trening...
Epoch 1/3
313/313 ━━━━━━━━━━━━━━━━━━━━ 96s 292ms/step - accuracy: 0.6747 - loss: 0.5746 - val_accuracy: 0.8566 - val_loss: 0.3354
Epoch 2/3
313/313 ━━━━━━━━━━━━━━━━━━━━ 89s 285ms/step - accuracy: 0.9078 - loss: 0.2407 - val_accuracy: 0.8644 - val_loss: 0.3429
Epoch 3/3
313/313 ━━━━━━━━━━━━━━━━━━━━ 90s 288ms/step - accuracy: 0.9308 - loss: 0.1793 - val_accuracy: 0.8754 - val_loss: 0.3510
